In [ ]:
from sdm.utils import set_project_wd
set_project_wd()

In [ ]:
import torch
from segmentation_models_pytorch import Unet
import rasterio as rio
# Define windows from block size
from rasterio.windows import Window
from typing import Tuple, Union
from pathlib import Path
import pandas as pd

In [ ]:
species_counts = pd.read_csv('data/processed/bat-species-counts.csv')
species_counts["count_weight"] = species_counts["total_records"] / species_counts["total_records"].sum()
species_counts

In [ ]:
ev_path = "data/processed/s2-dtm-100m.tif"
presences_path = "data/processed/bat-species-counts.tif"
sample_dir = Path("data/processed/dl-samples")
sample_dir.mkdir(exist_ok=True, parents=True)

In [ ]:
def generate_tif_blocks(tif_path:Union[str, Path], block_size:int, all_even:bool=False) -> list[Window]:
    windows = []
    with rio.open(tif_path) as src:
        height, width = src.height, src.width
        for i in range(0, height, block_size):
            for j in range(0, width, block_size):
                block_height = min(block_size, height - i)
                block_width = min(block_size, width - j)
                if all_even and (block_height == block_size) and (block_width == block_size):
                    windows.append(Window(j, i, block_width, block_height))
                elif not all_even:
                    windows.append(Window(j, i, block_width, block_height))
                else:
                    pass
    return windows

def get_tif_info(self, tif_path):
    """
    Get information about the tif file
    """
    with rio.open(tif_path) as src:
        return {"profile": src.profile, "transform": src.transform, "crs": src.crs, "channels": src.count, "height": src.height, "width": src.width}
